In [58]:
from plotnine import *
from scipy.spatial import distance
import pandas as pd
from dtw import *
import dtaidistance as dd
from tslearn import metrics
from tssearch import plot_alignment
from tssearch.search.query_search import time_series_search
from dtaidistance import dtw_visualisation as dtwvis
import matplotlib.pyplot as plt

In [64]:
time_series_udf_dataset = pd.read_csv("Organised/Time_series_udf_dataset.csv")
time_series_udf_dataset_normalised = pd.read_csv("Organised/Time_series_udf_dataset_normalized.csv")
time_series_udf_dataset_normalised_smooth = pd.read_csv("Organised/Time_series_udf_dataset_normalized_smooth.csv")
time_series_udf_dataset_smooth = pd.read_csv("Organised/Time_series_udf_dataset_smooth.csv")

time_series_aggregation = time_series_udf_dataset[time_series_udf_dataset["label"]=="aggregation"]
time_series_filtration = time_series_udf_dataset[time_series_udf_dataset["label"]=="filtration"]
time_series_filtration_aggregation = time_series_udf_dataset[time_series_udf_dataset["label"]=="filtration-aggregation"]
time_series_filtration_aggregation_join = time_series_udf_dataset[time_series_udf_dataset["label"]=="filtration-aggregation-join"]
time_series_filtration_join = time_series_udf_dataset[time_series_udf_dataset["label"]=="filtration-join"]

time_series_aggregation_normalised = time_series_udf_dataset[time_series_udf_dataset_normalised["label"]=="aggregation"]
time_series_filtration_normalised = time_series_udf_dataset[time_series_udf_dataset_normalised["label"]=="filtration"]
time_series_filtration_aggregation_normalised = time_series_udf_dataset[time_series_udf_dataset_normalised["label"]=="filtration-aggregation"]
time_series_filtration_aggregation_join_normalised = time_series_udf_dataset[time_series_udf_dataset_normalised["label"]=="filtration-aggregation-join"]
time_series_filtration_join_normalised = time_series_udf_dataset[time_series_udf_dataset_normalised["label"]=="filtration-join"]

time_series_aggregation_normalised_smooth = time_series_udf_dataset[time_series_udf_dataset_normalised_smooth["label"]=="aggregation"]
time_series_filtration_normalised_smooth = time_series_udf_dataset[time_series_udf_dataset_normalised_smooth["label"]=="filtration"]
time_series_filtration_aggregation_normalised_smooth = time_series_udf_dataset[time_series_udf_dataset_normalised_smooth["label"]=="filtration-aggregation"]
time_series_filtration_aggregation_join_normalised_smooth = time_series_udf_dataset[time_series_udf_dataset_normalised_smooth["label"]=="filtration-aggregation-join"]
time_series_filtration_join_normalised_smooth = time_series_udf_dataset[time_series_udf_dataset_normalised_smooth["label"]=="filtration-join"]

time_series_aggregation_smooth = time_series_udf_dataset[time_series_udf_dataset_smooth["label"]=="aggregation"]
time_series_filtration_smooth = time_series_udf_dataset[time_series_udf_dataset_smooth["label"]=="filtration"]
time_series_filtration_aggregation_smooth = time_series_udf_dataset[time_series_udf_dataset_smooth["label"]=="filtration-aggregation"]
time_series_filtration_aggregation_join_smooth = time_series_udf_dataset[time_series_udf_dataset_smooth["label"]=="filtration-aggregation-join"]
time_series_filtration_join_smooth = time_series_udf_dataset[time_series_udf_dataset_smooth["label"]=="filtration-join"]

FileNotFoundError: [Errno 2] No such file or directory: 'Organised/Time_series_udf_dataset_normalized_smooth.csv'

In [ ]:
def calculate_euclidean_distance(a_cpu, b_cpu, a_ram, b_ram):
    cpu_dst = distance.euclidean(a_cpu, b_cpu)
    ram_dst = distance.euclidean(a_ram, b_ram)
    return cpu_dst, ram_dst

def calculate_dtw_dtw_library(a_cpu, b_cpu, a_ram, b_ram):
    step_pattern = symmetric2

    alignment_cpu = dtw(a_cpu, b_cpu, step_pattern=step_pattern, keep_internals=True)
    alignment_ram = dtw(a_ram, b_ram, step_pattern=step_pattern, keep_internals=True)
    #alignment_cpu.plot(type="twoway", offset=80)
    #plt.show()
    return alignment_cpu.normalizedDistance, alignment_cpu.distance, alignment_ram.normalizedDistance, alignment_ram.distance

def calculate_dtw_dtaidistance_library(a_cpu, b_cpu, a_ram, b_ram):
    d_cpu = dd.dtw.distance(numpy.array(a_cpu), numpy.array(b_cpu), None, None, None, None, None, None, None, True)
    path = dd.dtw.warping_path(numpy.array(a_cpu), numpy.array(b_cpu))
    #dtwvis.plot_warping(numpy.array(a_cpu), numpy.array(b_cpu), path)

    d_ram = dd.dtw.distance(numpy.array(a_ram), numpy.array(b_ram))
    return d_cpu, d_ram

def calculate_lcss(a_cpu, b_cpu, a_ram, b_ram):
    lcss_path_cpu, sim_lcss_cpu = metrics.lcss_path(a_cpu, b_cpu, eps=2.0)
    lcss_path_ram, sim_lcss_ram = metrics.lcss_path(a_ram, b_ram, eps=1.5)
    return sim_lcss_cpu, sim_lcss_ram

def calculate_lcss2(a_cpu, b_cpu, a_ram, b_ram):
    dict_distances = {
        "elastic": {"Longest Common Subsequence": {
            "multivariate": "yes",
            "description": "",
            "function": "lcss",
            "parameters": {"eps": 1.5, "report": "distance"},
            "use": "yes"}
        }
    }
    sim_lcss_cpu = time_series_search(dict_distances, numpy.array(a_cpu), numpy.array(b_cpu), output=("number", 1))["Longest Common Subsequence"]["distance"]
    sim_lcss_ram = time_series_search(dict_distances, numpy.array(a_ram), numpy.array(b_ram), output=("number", 1))["Longest Common Subsequence"]["distance"]

    #result = time_series_search(dict_distances, numpy.array(a_ram), numpy.array(b_ram), output=("number", 1))
    #plt.figure()
    #plt.title("Longest Common Subsequence")
    #plot_alignment(a_ram, b_ram, result["Longest Common Subsequence"]["path"][0])
    return sim_lcss_cpu, sim_lcss_ram

## Creating comparison dataFrame

In [65]:
def calculate_distance_and_save_as_df(df, data, snapshot1, snapshot2, normalized=0, smoothed=0, data2=pd.DataFrame()):
    if data2.empty:
        data2 = data
    a = data[data["snapshot"] == snapshot1]
    b = data2[data2["snapshot"] == snapshot2]
    max_length = max(len(a.index), len(b.index))

    a_cpu = list(a['CPU'])
    b_cpu = list(b['CPU'])
    a_ram = list(a['RAM'])
    b_ram = list(b['RAM'])

    a_cpu = a_cpu + [0]*(max_length - len(a_cpu))
    b_cpu = b_cpu + [0]*(max_length - len(b_cpu))
    a_ram = a_ram + [0]*(max_length - len(a_ram))
    b_ram = b_ram + [0]*(max_length - len(b_ram))

    euc_cpu_dist, euc_ram_dist = calculate_euclidean_distance(a_cpu, b_cpu, a_ram, b_ram)
    dtw_cpu_dist_normalized, dtw_cpu_dist, dtw_ram_dist_normalized, dtw_ram_dist = \
        calculate_dtw_dtw_library(a_cpu, b_cpu, a_ram, b_ram)
    dtaidistance_cpu, dtaidistance_ram = calculate_dtw_dtaidistance_library(a_cpu, b_cpu, a_ram, b_ram)
    lcss_distance_cpu, lcss_distance_ram = calculate_lcss(a_cpu, b_cpu, a_ram, b_ram)
    new_cpu_row = {
        'dataType' : 'CPU',
        'function1' : a['label'],
        'snapshot1' : snapshot1,
        'udf1' : a['udf'].max(),
        'function2' : b['label'],
        'snapshot2': snapshot2,
        'udf2' : b['udf'].max(),
        'euclidean': euc_cpu_dist,
        'dtw-python-distance': dtw_cpu_dist,
        'dtw-python-normalizedDistance': dtw_cpu_dist_normalized,
        'dtaidistance': dtaidistance_cpu,
        'lcss_similarity': lcss_distance_cpu,
        'is_normalized': normalized,
        'is_smoothed': smoothed
    }
    new_ram_row = {
        'dataType' : 'RAM',
        'function1' : function1,
        'snapshot1' : snapshot1,
        'udf1' : a['udf'].max(),
        'function2' : function2,
        'snapshot2': snapshot2,
        'udf2' : b['udf'].max(),
        'euclidean': euc_ram_dist,
        'dtw-python-distance': dtw_ram_dist,
        'dtw-python-normalizedDistance': dtw_ram_dist_normalized,
        'dtaidistance': dtaidistance_ram,
        'lcss_similarity': lcss_distance_ram,
        'is_normalized': normalized,
        'is_smoothed': smoothed
    }
    df = df.append(new_cpu_row, ignore_index = True)
    df = df.append(new_ram_row, ignore_index = True)
    return df

In [ ]:
columns = ['dataType', 'function1', 'snapshot1','function2', 'snapshot2', 'euclidean', 'dtw-python-distance', 'dtw-python-normalizedDistance',
           'dtaidistance', 'lcss_similarity', 'is_normalized', 'is_smoothed']
dataFrame = pd.DataFrame([], columns = columns)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0,0)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0,5)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0,150)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0, 0, 0, 0, time_series_filtration)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0, 0, 0, 0, time_series_filtration_aggregation)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0, 0, 0, 0, time_series_filtration_aggregation_join)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation, 0, 0, "aggregation", "filtration-join", 0, 0, time_series_filtration_join)

dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0,0, "aggregation", "aggregation", 0, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0,5, "aggregation", "aggregation", 0, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0,150, "aggregation", "aggregation", 0, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0, 0, "aggregation", "filtration", 0, 1, time_series_filtration_smooth)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0, 0, "aggregation", "filtration-aggregation", 0, 1, time_series_filtration_aggregation_smooth)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0, 0, "aggregation", "filtration-aggregation-join", 0, 1, time_series_filtration_aggregation_join_smooth)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_smooth, 0, 0, "aggregation", "filtration-join", 0, 1, time_series_filtration_join_smooth)

dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0,0, "aggregation", "aggregation", 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0,5, "aggregation", "aggregation", 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0,150, "aggregation", "aggregation", 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0, 0, "aggregation", "filtration", 1, 0, time_series_filtration_normalised)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0, 0, "aggregation", "filtration-aggregation", 1, 0, time_series_filtration_aggregation_normalised)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0, 0, "aggregation", "filtration-aggregation-join", 1, 0, time_series_filtration_aggregation_join_normalised)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised, 0, 0, "aggregation", "filtration-join", 1, 0, time_series_filtration_join_normalised)

dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0,0, "aggregation", "aggregation", 1, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0,5, "aggregation", "aggregation", 1, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0,150, "aggregation", "aggregation", 1, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0, 0, "aggregation", "filtration", 1, 1, time_series_filtration_normalised_smooth)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0, 0, "aggregation", "filtration-aggregation", 1, 1, time_series_filtration_aggregation_normalised_smooth)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0, 0, "aggregation", "filtration-aggregation-join", 1, 1, time_series_filtration_aggregation_join_normalised_smooth)
dataFrame = calculate_distance_and_save_as_df(dataFrame, time_series_aggregation_normalised_smooth, 0, 0, "aggregation", "filtration-join", 1, 1, time_series_filtration_join_normalised_smooth)

In [41]:
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0,0, "aggregation", "aggregation")
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0,5, "aggregation", "aggregation")
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0,150, "aggregation", "aggregation")
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0, 0, "aggregation", "filtration", 0, 0, joined_filtration_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0, 0, "aggregation", "filtration-aggregation", 0, 0, joined_aggregation_filtration_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0, 0, "aggregation", "filtration-aggregation-join", 0, 0, joined_filtration_aggregation_join_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_all, 0, 0, "aggregation", "filtration-join", 0, 0, joined_filtration_join_all)

dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0,0, "aggregation", "aggregation", 0, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0,5, "aggregation", "aggregation", 0, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0,150, "aggregation", "aggregation", 0, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0, 0, "aggregation", "filtration", 0, 1, joined_filtration_smooth_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0, 0, "aggregation", "filtration-aggregation", 0, 1, joined_aggregation_filtration_smooth_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0, 0, "aggregation", "filtration-aggregation-join", 0, 1, joined_filtration_aggregation_join_smooth_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, joined_filtration_smooth_all, 0, 0, "aggregation", "filtration-join", 0, 1, joined_filtration_join_smooth_all)

dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_filtration, 0,0, "aggregation", "aggregation", 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_filtration, 0,5, "aggregation", "aggregation", 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_filtration, 0,150, "aggregation", "aggregation", 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_filtration, 0, 0, "aggregation", "filtration", 1, 0, normalized_joined_filtration)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_filtration, 0, 0, "aggregation", "filtration-aggregation", 1, 0, normalized_joined_aggregation_filtration)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_filtration, 0, 0, "aggregation", "filtration-aggregation-join", 1, 0, normalized_joined_filtration_aggregation_join)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation, 0, 0, "aggregation", "filtration-join", 1, 0, normalized_joined_filtration_join)

dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0,0, "aggregation", "aggregation", 1, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0,5, "aggregation", "aggregation", 1, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0,150, "aggregation", "aggregation", 1, 1)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0, 0, "aggregation", "filtration", 1, 1, normalized_joined_filtration_smooth_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0, 0, "aggregation", "filtration-aggregation", 1, 1, normalized_joined_aggregation_filtration_smooth_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0, 0, "aggregation", "filtration-aggregation-join", 1, 1, normalized_joined_filtration_aggregation_join_smooth_all)
dataFrame = calculate_distance_and_save_as_df(dataFrame, normalized_joined_aggregation_smooth_all, 0, 0, "aggregation", "filtration-join", 1, 1, normalized_joined_filtration_join_smooth_all)

In [43]:
dataFrame.to_csv("comparison_distance.csv", index=False)
dataFrame


,dataType,function1,snapshot1,function2,snapshot2,euclidean,dtw-python-distance,dtw-python-normalizedDistance,dtaidistance,lcss_similarity,is_normalized,is_smoothed,udf1,udf2
0,CPU,aggregation,0,aggregation,0,0.000000,0.000000,0.000000,0.000000,1.000000,0,0,avgNetProfitGroupedBySoldDate,avgNetProfitGroupedBySoldDate
1,RAM,aggregation,0,aggregation,0,0.000000,0.000000,0.000000,0.000000,1.000000,0,0,avgNetProfitGroupedBySoldDate,avgNetProfitGroupedBySoldDate
2,CPU,aggregation,0,aggregation,5,949.151274,4444.600000,3.919400,342.577801,0.802469,0,0,avgNetProfitGroupedBySoldDate,avgNetProfitGroupedBySoldDate
3,RAM,aggregation,0,aggregation,5,12.602381,23.500000,0.020723,8.254090,0.989418,0,0,avgNetProfitGroupedBySoldDate,avgNetProfitGroupedBySoldDate
4,CPU,aggregation,0,aggregation,150,901.985831,4627.400000,4.109591,368.190807,0.793961,0,0,avgNetProfitGroupedBySoldDate,avgWholeSaleCostGroupedBySoldDate
5,RAM,aggregation,0,aggregation,150,17.935440,138.300000,0.122824,7.727872,0.992895,0,0,avgNetProfitGroupedBySoldDate,avgWholeSaleCostGroupedBySoldDate
6,CPU,aggregation,0,filtration,0,1165.670515,4270.100000,3.792274,384.745032,0.769094,0,0,avgNetProfitGroupedBySoldDate,filterCatalogSalesWhereProfitNegative
7,RAM,aggregation,0,filtration,0,205.663998,1866.800000,1.657904,84.147965,0.261101,0,0,avgNetProfitGroupedBySoldDate,filterCatalogSalesWhereProfitNegative
8,CPU,aggregation,0,filtration-aggregation,0,1234.195706,4468.600000,3.214820,383.003877,0.812950,0,0,avgNetProfitGroupedBySoldDate,avgNetProfitGroupedBySoldDateWhereProfitNegative
9,RAM,aggregation,0,filtration-aggregation,0,137.442133,1734.000000,1.247482,72.291424,0.220144,0,0,avgNetProfitGroupedBySoldDate,avgNetProfitGroupedBySoldDateWhereProfitNegative
